# Lab 2 - Create an agent with a Knowledge Base and an Action Group

In this notebook, we will be creating a Amazon Bedrock Agent responsible for handling environment access tickets from employee's. We will also associate an Amazon Bedrock Knowledge Base giving the agent ability to decide using companies policies whether to auto-resolve ticket or assign ticket to Environment owner.

1. Notebook setup
2. Create Lambda Function
3. Create Amazon Bedrock Agents
4. Create Agent Action Group
5. Associate Amazon Bedrock Knowledge Base with Agent
6. Invoke Ticket Agent

![architecture-agent](images/architecture-agent.png)

## 1. Notebook setup


In [ ]:
%store -r

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
from utility.knowledgebase import interactive_sleep

print(boto3.__version__)

In [ ]:
import logging

# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [ ]:
session = boto3.session.Session()
region_name = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region_name, account_id

In [ ]:
agent_name = "ticket-assistant"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_knowledgebase_allow_policy_name = f"{agent_name}-kb-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent for resolve Employee tickets"
agent_instruction = """You are a ticket resolving agent responsible for handling environment access tickets from employee's. 
Decide using companies policies whether to auto-resolve ticket or assign ticket to Environment owner.
"""

kb_description = "This knowledge base consists of company policies regarding providing environment access to employees."

agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## 2. Create Lambda Function


We will now create a Lambda function that interacts with the `UserTable`, `EnvironmentTable`, `UserAccessTable`, and `TicketTable` Amazon DynamoDB tables. To do so we will:

Create the lambda_function.py file which contains the logic for our Lambda function
Create the IAM role for our Lambda function
Create the Lambda function infrastructure with the required permissions

In [ ]:
%%writefile lambda_function.py

import json
import boto3

dynamodb = boto3.resource('dynamodb')

UserDynamoDBTable = dynamodb.Table('UserTable')
EnvironmentDynamoDBTable = dynamodb.Table('EnvironmentTable')
UserAccessDynamoDBTable = dynamodb.Table('UserAccessTable')
TicketDynamoDBTable = dynamodb.Table('TicketTable')

def checkEmployeeAccess(employeeId, environmentId):
    response = UserAccessDynamoDBTable.get_item(Key={'employeeId': employeeId, 'environmentId': environmentId})
    if 'Item' in response:
        return True
    else:
        return False

def getEnvironmentOwner(environmentId):
    response = EnvironmentDynamoDBTable.get_item(Key={'environmentId': environmentId})
    if 'Item' in response:
        item = response['Item']
        owner = item['managing_entity']
        return f"Owner of Environment {environmentId} is {owner}."
    else:
        return f"Environment {environmentId} does not exist."
        

def getEmployeeManager(employeeId):
    response = UserDynamoDBTable.get_item(Key={'employeeId': employeeId})
    if 'Item' in response:
        item = response['Item']
        manager = item['manager']
        return manager
    

def giveAccess(employeeId, environmentId):
    response = UserAccessDynamoDBTable.put_item(Item={'employeeId': employeeId, 'environmentId': environmentId, 'description': 'Assigned by ticket resolving bot!'})

def autoResolveTicket(employeeId, ticketId, description):

    # Define the key(s) of the item you want to update
    key = {
        'ticketId': ticketId,
        'employeeId': employeeId
    }

    # Define the attribute updates
    update_expression = 'SET assignStatus = :newAssignStatus,communication = :newCommunication'
    expression_attribute_values = {
        ':newAssignStatus': 'auto-resolved',
        ':newCommunication': description
    }

    # Update the item
    response = TicketDynamoDBTable.update_item(
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeValues=expression_attribute_values,
        ReturnValues='UPDATED_NEW'
    )

def assignTicketToEnvironmentOwner(employeeId, ticketId, owner, instructions):
    # Define the key(s) of the item you want to update
    key = {
        'ticketId': ticketId,
        'employeeId': employeeId
    }

    # Define the attribute updates
    update_expression = 'SET assignStatus = :newAssignStatus,communication = :newCommunication'
    expression_attribute_values = {
        ':newAssignStatus': f'assigned to {owner}',
        ':newCommunication': instructions
    }

    # Update the item
    response = TicketDynamoDBTable.update_item(
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeValues=expression_attribute_values,
        ReturnValues='UPDATED_NEW'
    )

def lambda_handler(event, context):
    agent = event['agent']
    actionGroup = event['actionGroup']
    parameters = event.get('parameters', [])
    requestBody = event.get('requestBody', {})
    function = event['function']
    session_attributes = event.get('sessionAttributes', {})
    employeeId = session_attributes.get('employeeId', '')
    ticketId = session_attributes.get('ticketId', '')


    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }

    if function == 'checkEmployeeAccess':
        environmentId = None
        for param in parameters:
            if param["name"] == "environmentId":
                environmentId = param["value"]

        if employeeId and environmentId:
            access = checkEmployeeAccess(employeeId, environmentId)
            if access:
                responseBody =  {
                    'TEXT': {
                        "body": f"User {employeeId} has access to Environment {environmentId}"
                    }
                }
            else:
                responseBody =  {
                    'TEXT': {
                        "body": f"User {employeeId} does not have access to Environment {environmentId}"
                    }
                }

        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}
            
       
        
    elif function == 'getEnvironmentOwner':
        environmentId = None
        for param in parameters:
            if param["name"] == "environmentId":
                environmentId = param["value"]
            
        if environmentId:
            response = getEnvironmentOwner(environmentId)
            responseBody =  {
                'TEXT': {
                    "body": response
                }
            }
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters environmentId'}}

    elif function == 'getEmployeeManager':
        if employeeId:
            manager = getEmployeeManager(employeeId)
            responseBody =  {
                'TEXT': {
                    "body": f"Manager of employee {employeeId} is {manager}"
                }
            }
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters employeeId'}} 
    
    elif function == 'giveAccess':
        environmentId = None
        for param in parameters:
            if param["name"] == "environmentId":
                environmentId = param["value"]

        if employeeId and environmentId:
            manager = giveAccess(employeeId, environmentId)
            responseBody =  {
                'TEXT': {
                    "body": f"Access to {employeeId} completed!"
                }
            }
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'autoResolveTicket':
        description = None
        for param in parameters:
            if param["name"] == "description":
                description = param["value"]

        if employeeId and ticketId and description:
            _ = autoResolveTicket(employeeId, ticketId, description)
            responseBody =  {
                'TEXT': {
                    "body": f"Ticket auto-resolved successfully"
                }
            }
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}   
    
    elif function == 'assignTicketToEnvironmentOwner':
        owner = None
        instructions = None
        for param in parameters:
            if param["name"] == "owner":
                owner = param["value"]
            if param["name"] == "instructions":
                instructions = param["value"]

        if employeeId and ticketId and owner and instructions:
            _ = assignTicketToEnvironmentOwner(employeeId, ticketId, owner, instructions)
            responseBody =  {
                'TEXT': {
                    "body": f"Ticket assigned successfully"
                }
            }
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}       
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Next let's create the lambda IAM role and policy to invoke a Bedrock model


In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    interactive_sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

dynamodb_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Statement1",
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:UpdateItem",
                "dynamodb:PutItem"
            ],
            "Resource": [
                f"arn:aws:dynamodb:{region_name}:{account_id}:table/UserTable",
                f"arn:aws:dynamodb:{region_name}:{account_id}:table/EnvironmentTable",
                f"arn:aws:dynamodb:{region_name}:{account_id}:table/UserAccessTable",
                f"arn:aws:dynamodb:{region_name}:{account_id}:table/TicketTable"
            ]
        }
    ]
}

db_policy_name = f'DynamoDBPolicy-{agent_name}-{suffix}'

db_policy = iam_client.create_policy(
    PolicyName=db_policy_name,
    PolicyDocument=json.dumps(dynamodb_policy_document),
    Description='Policy for accessing DynamoDB tables',
)

iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=db_policy["Policy"]["Arn"]
)
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

We can now package the lambda function to a Zip file and create the lambda function using boto3

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

In [ ]:
lambda_function_arn = lambda_function['FunctionArn']
%store lambda_function_arn lambda_function_name lambda_function_role db_policy_name

## 3. Create Amazon Bedrock Agent


### 3.1 Create Amazon Bedrock Agent Role and Policy

We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it.



In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region_name}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

Since the agent will also invoke Amazon Bedrock Knowledge Base, we need an allow policy for retrieve action.

In [ ]:
bedrock_agent_knowledgebase_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockKnowledgeBasePolicy",
            "Effect": "Allow",
            "Action": "bedrock:Retrieve",
            "Resource": [
                f"arn:aws:bedrock:{region_name}:{account_id}:knowledge-base/{kb_id}"
            ]
        }
    ]
}

bedrock_knowledgebase_policy_json = json.dumps(bedrock_agent_knowledgebase_allow_policy_statement)

agent_kb_policy = iam_client.create_policy(
    PolicyName=agent_knowledgebase_allow_policy_name,
    PolicyDocument=bedrock_knowledgebase_policy_json
)

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
interactive_sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_policy['Policy']['Arn']
)
agent_bedrock_policy_arn = agent_bedrock_policy['Policy']['Arn']

In [ ]:
%store agent_role_name agent_bedrock_allow_policy_name agent_knowledgebase_allow_policy_name

## 3.2 Create Amazon Bedrock Agent

Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.


In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)
response

In [ ]:
agent_id = response['agent']['agentId']
agent_id

In [ ]:
%store agent_id

### 3.3 Allowing Agent to invoke Action Group Lambda

Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created.

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region_name}:{account_id}:agent/{agent_id}",
)

## 4. Create Agent Action Group


We will now create three agent action groups that uses the lambda function created earlier. These three are:

1. EnvironmentInformationAPI
2. EmployeeInformationAPI
3. TicketAPI

The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides the functionality to create action groups. We have conceptually seperated the action groups. Best practices for building robust generative AI applications with Amazon Bedrock Agents can be found here, [part-1](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-1/) and [part-2](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-2/).

We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

In [ ]:

agent_action_group_name_env = "EnvironmentInformationAPI"
agent_action_group_description_env = "API for Environment"

agent_functions_env = [
    {
        'name': 'getEnvironmentOwner',
        'description': 'Get the owner of the environment',
        'parameters': {
            "environmentId": {
                "description": "EnvironmentId of the environment",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'giveAccess',
        'description': 'Give an employee access to the environment',
        'parameters': {
            "environmentId": {
                "description": "EnvironmentId of the environment",
                "required": True,
                "type": "string"
            }
        }
    },
]

# Pause to make sure agent is created
interactive_sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response_env = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name_env,
    functionSchema={
        'functions': agent_functions_env
    },
    description=agent_action_group_description_env
)

action_group_id_env = agent_action_group_response_env['agentActionGroup']['actionGroupId']
action_group_name_env = agent_action_group_response_env['agentActionGroup']['actionGroupName']

In [ ]:
%store action_group_id_env action_group_name_env agent_functions_env

In [ ]:
agent_action_group_name_employee = "EmployeeInformationAPI"
agent_action_group_description_employee = "API for Information"

agent_functions_employee = [
    {
        'name': 'checkEmployeeAccess',
        'description': 'Check if employee has access to the Environment',
        'parameters': {
            "environmentId": {
                "description": "EnvironmentId of the environment",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'getEmployeeManager',
        'description': 'Get the manager of an employee',
    },
]
# Configure and create an action group here:
agent_action_group_response_employee = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name_employee,
    functionSchema={
        'functions': agent_functions_employee
    },
    description=agent_action_group_description_employee
)

action_group_id_employee = agent_action_group_response_employee['agentActionGroup']['actionGroupId']
action_group_name_employee = agent_action_group_response_employee['agentActionGroup']['actionGroupName']

In [ ]:
%store action_group_id_employee action_group_name_employee agent_functions_employee

In [ ]:

agent_action_group_name_ticket = "TicketAPI"
agent_action_group_description_ticket = "API for handing tickets"

agent_functions_ticket = [
    {
        'name': 'autoResolveTicket',
        'description': 'Auto-resolve the ticket',
        'parameters': {
            "description": {
                "description": "Describe why was the ticket auto-resolved",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'assignTicketToEnvironmentOwner',
        'description': 'Assign the ticket to Environment owner',
        'parameters': {
            "owner": {
                "description": "Owner of the environment",
                "required": True,
                "type": "string"
            },
            "instructions": {
                "description": "Clear instructions on how to grant access and best practices for managing environment permissions.",
                "required": True,
                "type": "string"
            }
        }
    },
]

# Configure and create an action group here:
agent_action_group_response_ticket = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name_ticket,
    functionSchema={
        'functions': agent_functions_ticket
    },
    description=agent_action_group_description_ticket
)

action_group_id_ticket = agent_action_group_response_ticket['agentActionGroup']['actionGroupId']
action_group_name_ticket = agent_action_group_response_ticket['agentActionGroup']['actionGroupName']

In [ ]:
%store action_group_id_ticket action_group_name_ticket agent_functions_ticket

## 5. Associate Amazon Bedrock Knowledge Base with Agent


In [ ]:
agent_kb = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=kb_description,
    knowledgeBaseId=kb_id 
)

## 6. Invoke Ticket Agent

### 6.1 Preparing agent

Let's create a DRAFT version of the agent that can be used for internal testing.



In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
# Pause to make sure agent is prepared
interactive_sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

### 6.2 Invoke Agent

In [ ]:
from utility.ticket import create_ticket, display_table

<div class="alert alert-block alert-info">
<b>Information:</b> create_ticket and display_table utility functions

- <b>create_ticket</b>: Creates a ticket in ticket table
- <b>display_table</b>: Displays the current items in ticket table
</div>

#### 6.2.1 DynamoDB tables

In [ ]:
display_table('UserTable')

In [ ]:
display_table('EnvironmentTable')

In [ ]:
display_table('UserAccessTable')

In [ ]:
display_table('TicketTable')

#### 6.2.2 Create ticket

You can see above that `employee 111` does not have access to `environment 1`

In [ ]:
ticket = """
Title: Request Environment Access
EnvironmentId: 1
Buisness Justification: Need to test new features on Quicksight Dashboard
Access duration: 8 days
Access Type: Read
"""
employeeId = '111'

In [ ]:
ticketId = create_ticket(ticket, employeeId)


In [ ]:
display_table('TicketTable')

Ticket is created but unassigned.

In [ ]:
inputText = f"""
Please either auto-resolve the ticket or assign it to environment owner:

<ticket>
{ticket}
</ticket>

Ensure to resolve the ticket by calling TicketAPI. 
"""

In [ ]:
sessionId = str(uuid.uuid1())
response = bedrock_agent_runtime_client.invoke_agent(
                  inputText=inputText,
                  agentId=agent_id, # Agent ID
                  agentAliasId=agent_alias_id, 
                  sessionId=sessionId,
                  enableTrace=True, 
                  endSession=False,
                  sessionState={
                      "sessionAttributes": {
                          "employeeId": employeeId, # sessionAttributes
                          "ticketId": ticketId # ticketId
                      },
                      "knowledgeBaseConfigurations": [
                        {
                            'knowledgeBaseId': kb_id,
                            'retrievalConfiguration': {
                                'vectorSearchConfiguration': {
                                    'numberOfResults': 3,
                                    'overrideSearchType': 'HYBRID'
                                }
                            }
                        },
                    ]
                  }
              )

In [ ]:
agent_answer = ""

In [ ]:
%%time
event_stream = response['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
print(agent_answer)

<div class="alert alert-block alert-info">
<b>Information:</b> Access to the employee can be auto-assigned if all the given scenarios are true:

1. The environment is owned by the manager of the employee.
2. The employee has asked for less than 30 days of access.
3. The access type is not Admin.
</div>

Lets evaluate the above scenarios for this test case:
1. `@sam` is the owner of `environment 1` and `employee 111`'s manager is also `@sam`.
2. Employee has asked for 8 days of access.
3. Access type is read.

This ticket should be assigned to `@susi`. 

In [ ]:
display_table('UserAccessTable')

You can see above that `employee 111` now has access to `environment 1`

In [ ]:
display_table('TicketTable')

Ticket is now auto-resolved.

<div class="alert alert-block alert-success">
<b>Disclaimer:</b> Please note that the output of Large Language Models (LLMs) is non-deterministic, meaning that the responses generated may vary each time the model is run. Due to this inherent nature of LLMs, there is a possibility of observing undesirable or unexpected behavior. If such a situation arises, we recommend running the above workflow to obtain a new response.

To enhance the resilience and robustness of this application, we strongly recommend implementing the best practices outlined in the following resources:

1. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 1](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-1/)**
2. **[Best Practices for Building Robust Generative AI Applications with Amazon Bedrock Agents - Part 2](https://aws.amazon.com/blogs/machine-learning/best-practices-for-building-robust-generative-ai-applications-with-amazon-bedrock-agents-part-2/)**
</div>


<div class="alert alert-block alert-warning">
<b>Next steps:</b> Proceed to the next labs to test event-driven workflow. Remember to run the CLEANUP notebook at the end of your session.
</div>